<b>Step1</b>:Import Beautiful Soup library. Scrape Wikipedia page to get Postal codes for Toronto from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [99]:
from bs4 import BeautifulSoup

<b>Step 2: </b> Get scraped data as dataframe

In [100]:
import pandas as pd
import requests
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(url)
soup = BeautifulSoup(response.text,'lxml')     
data = []
table = soup.find('table', attrs={'class':'wikitable sortable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) 
# convert into dataframe with specified columns
new_table = pd.DataFrame(data, columns = ['PostalCode','Burough','Neighborhood'])

<b>Step 3: </b> Drop unassigned Buroughs

In [101]:
new_table.drop(new_table.index[0],inplace = True) # drop first empty row
new_table.drop(new_table[new_table['Burough']== 'Not assigned'].index, inplace = True)

<b>Step 4: </b> Assign Burough name to Unassigned Neighborhood

In [102]:
new_table.Neighborhood[new_table.Neighborhood == 'Not assigned'] = new_table.Burough[new_table.Neighborhood == 'Not assigned']
new_table = new_table.reset_index(drop = True)

<b>Step 5: </b> Concat Neighborhoods with same postal code

In [103]:
test_table = pd.DataFrame(new_table)
for i in test_table.index:
        val = test_table.get_value(i,'PostalCode')
        neigh = test_table.get_value(i,'Neighborhood')
        neigh_prev = neigh
        prev_index = 0
        for j in test_table.index:            
            if test_table.get_value(j,'PostalCode') == val and test_table.get_value(j,'Neighborhood') != neigh_prev:                 
                neigh = neigh + ', ' + test_table.get_value(j,'Neighborhood')
            elif test_table.get_value(j,'PostalCode') == val and test_table.get_value(j,'Neighborhood') == neigh:
                neigh_prev = test_table.get_value(j,'Neighborhood')
                neigh = test_table.get_value(j,'Neighborhood')                      
       
       
        test_table.set_value(i,'Neighborhood',neigh)
        
test_table.drop_duplicates(subset='PostalCode', keep='first', inplace= True)        
test_table.reset_index(drop = True)

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.
/home/jupyterlab

,PostalCode,Burough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


<b>Step 5: </b> Display dataframe in prescribed format

In [106]:
test_table

,PostalCode,Burough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,"Rouge, Malvern"
10,M3B,North York,Don Mills North
11,M4B,East York,"Woodbine Gardens, Parkview Hill"
13,M5B,Downtown Toronto,"Ryerson, Garden District"


<b>Step 6: </b> Display shape of dataframe

In [107]:
test_table.shape

(103, 3)